In [1]:
import numpy as np
from scipy import sparse
import json
from edward.models import Poisson
from edward.models import Gamma as IGR
import tensorflow as tf
import gc

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
with open('package_id_dict.json', 'r') as f:
    package_id_dict = json.load(f)

In [3]:
id_package_dict = {x: n for n, x in package_id_dict.items()}

In [4]:
with open('manifest_id_dict.json', 'r') as f:
    manifest_id_dict = json.load(f)

In [5]:
manifest_id_dict = {n: set(x) for n, x in manifest_id_dict.items()}

In [6]:
sparse_matrix = sparse.load_npz('item_result_matrix.npz')
beta = sparse_matrix.toarray()
del(sparse_matrix)
sparse_matrix = sparse.load_npz('user_result_matrix.npz')
theta = sparse_matrix.toarray()
del(sparse_matrix)
manifests = theta.shape[0]
packages = beta.shape[0]

In [7]:
manifests

9523

In [8]:
beta.shape

(12405, 13)

In [9]:
theta.shape

(9523, 13)

In [11]:
# Known ID
# input_list = ["org.slf4j:slf4j-api",
#                       "org.apache.commons:commons-lang3",
#                       "com.fasterxml.jackson.core:jackson-annotations",
#                       "com.fasterxml.jackson.core:jackson-core",
#                       "org.apache.calcite.avatica:avatica-core",
#                       "org.apache.calcite:calcite-linq4j",
#                       "commons-dbcp:commons-dbcp",
#                       "com.google.code.findbugs:jsr305",
#                       "org.codehaus.janino:janino",
#                       "com.fasterxml.jackson.core:jackson-databind",
#                       "org.codehaus.janino:commons-compiler",
#                       "com.esri.geometry:esri-geometry-api",
#                       "com.google.guava:guava",
#                       "net.hydromatic:aggdesigner-algorithm"]

In [25]:
# Unknown ID
input_list = ["io.vertx:vertx-core", "io.vertx:vertx-web"]

In [26]:
input_id_set = set()
unknown_list = []
for package_name in input_list:
    p_id = package_id_dict.get(package_name, -1)
    if p_id == -1:
        unknown_list.append(package_name)
    else:
        input_id_set.add(p_id)

In [27]:
input_id_set

{7033, 7466}

In [28]:
for n, x in manifest_id_dict.items():
    if x == input_id_set:
        break
else:
    n = -1

In [29]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()

In [30]:
n = int(n)
n

-1

In [31]:
if n == -1:
    theta_test = []
    a = a_c = c = c_c = 0.5
    b_c = d_c = 0.99
    K = 13
    epsilon = IGR(a_c, a_c/b_c).eval()
    theta_func = IGR(a, epsilon).eval()
    theta_test.append(epsilon*theta_func)
    theta_test = np.array(theta_test*K, dtype=float)
    result = Poisson(theta_test)
    result = result.prob(beta).eval()
else:
    result = Poisson(theta[n])
    result = result.prob(beta).eval()

In [32]:
result4 = np.zeros([12405])
for i in range(12405):
    result4[i] = result[i].mean()

In [33]:
highest_indices = result4.argsort()[-5:-1]
companion_recommendation = []
for package_id in highest_indices:
    prob_score = result4[package_id]
    package_name = id_package_dict[package_id]
    recommendation = {
        "cooccurrence_probability": prob_score * 100,
        "package_name": package_name,
        "topic_list": []
    }
    companion_recommendation.append(recommendation)

In [34]:
companion_recommendation

[{'cooccurrence_probability': 92.77060753421064,
  'package_name': 'uk.co.bluegecko:core-server',
  'topic_list': []},
 {'cooccurrence_probability': 92.77060753421064,
  'package_name': 'com.smartitengineering.smart-exim:smart-exim-xml-impl',
  'topic_list': []},
 {'cooccurrence_probability': 92.77060753421064,
  'package_name': 'org.apache.kafka:kafka_2.11',
  'topic_list': []},
 {'cooccurrence_probability': 92.77060753421064,
  'package_name': 'org.xwiki.platform:xwiki-platform-store-filesystem-attachments',
  'topic_list': []}]